Nesse notebook iremos realizar o treino do nosso modelo.

In [3]:
import json
from collections import Counter, OrderedDict
import csv
import pandas as pd
import os

In [23]:
PATH_INPUT = os.getcwd() + '/inputs/'
PATH_INPUT

'/Users/luizeduardocartolano/Dropbox/DUDU/Unicamp/IC/MC906/workspace/got-predictor/inputs/'

Carrega o arquivo json com todos os personagens de Game Of Thrones

In [31]:
df_2 = pd.read_csv(PATH_INPUT+'character-predictions.csv', usecols=['name','numDeadRelations', 'popularity','isAlive'])
df_2

,name,numDeadRelations,popularity,isAlive
0,Viserys II Targaryen,11,0.605351,0
1,Walder Frey,1,0.896321,1
2,Addison Hill,0,0.267559,1
3,Aemma Arryn,0,0.183946,0
4,Sylva Santagar,0,0.043478,1
5,Tommen Baratheon,5,1.000000,1
6,Valarr Targaryen,0,0.431438,0
7,Viserys I Targaryen,5,0.678930,0
8,Wilbert,0,0.006689,0
9,Wilbert Osgrey,0,0.020067,1


In [7]:
with open(PATH_INPUT+"characters.json") as input_file1:
    characters_data = json.load(input_file1)
characters_data

{'characters': [{'characterName': 'Addam Marbrand',
   'characterLink': '/character/ch0305333/',
   'actorName': 'B.J. Hogg',
   'actorLink': '/name/nm0389698/'},
  {'characterName': 'Aegon Targaryen',
   'houseName': 'Targaryen',
   'royal': True,
   'parents': ['Elia Martell', 'Rhaegar Targaryen'],
   'siblings': ['Rhaenys Targaryen', 'Jon Snow'],
   'killedBy': ['Gregor Clegane']},
  {'characterName': 'Aeron Greyjoy',
   'houseName': 'Greyjoy',
   'characterImageThumb': 'https://images-na.ssl-images-amazon.com/images/M/MV5BNzI5MDg0ZDAtN2Y2ZC00MzU1LTgyYjQtNTBjYjEzODczZDVhXkEyXkFqcGdeQXVyNTg0Nzg4NTE@._V1._SX100_SY140_.jpg',
   'characterImageFull': 'https://images-na.ssl-images-amazon.com/images/M/MV5BNzI5MDg0ZDAtN2Y2ZC00MzU1LTgyYjQtNTBjYjEzODczZDVhXkEyXkFqcGdeQXVyNTg0Nzg4NTE@._V1_.jpg',
   'characterLink': '/character/ch0540081/',
   'actorName': 'Michael Feast',
   'actorLink': '/name/nm0269923/',
   'siblings': ['Balon Greyjoy', 'Euron Greyjoy'],
   'nickname': 'Damphair'},
  {'cha

Realiza a primeira parte do nosso treino, que reune um dicionario com o nome do personagem, numero de personagens que ele matou e se está vivo ou morto

In [16]:
training_set_part1 = []
for character in characters_data['characters']:
    training_row = OrderedDict()

    training_row['character_name'] = str(character['characterName']) if 'characterName' in character else ' '
    training_row['kill_count'] = len(character['killed']) if 'killed' in character else 0
    training_row['dead'] = 1 if 'killedBy' in character else 0

    training_set_part1.append(training_row)
    
training_set_part1

[OrderedDict([('character_name', 'Addam Marbrand'),
              ('kill_count', 0),
              ('dead', 0)]),
 OrderedDict([('character_name', 'Aegon Targaryen'),
              ('kill_count', 0),
              ('dead', 1)]),
 OrderedDict([('character_name', 'Aeron Greyjoy'),
              ('kill_count', 0),
              ('dead', 0)]),
 OrderedDict([('character_name', 'Aerys II Targaryen'),
              ('kill_count', 2),
              ('dead', 1)]),
 OrderedDict([('character_name', 'Akho'), ('kill_count', 0), ('dead', 1)]),
 OrderedDict([('character_name', 'Alliser Thorne'),
              ('kill_count', 1),
              ('dead', 1)]),
 OrderedDict([('character_name', 'Alton Lannister'),
              ('kill_count', 0),
              ('dead', 1)]),
 OrderedDict([('character_name', 'Alys Karstark'),
              ('kill_count', 0),
              ('dead', 0)]),
 OrderedDict([('character_name', 'Amory Lorch'),
              ('kill_count', 1),
              ('dead', 1)]),
 OrderedDic

Carrega o arquivo com todos os episódios da série (até o ep 2 da season 8). O arquivo contém a descrição de todas as cenas de cada episódio.

In [12]:
with open(PATH_INPUT+"episodes.json") as input_file2:
        episodes_data = json.load(input_file2)
episodes_data

{'episodes': [{'seasonNum': 1,
   'episodeNum': 1,
   'episodeTitle': 'Winter Is Coming',
   'episodeLink': '/title/tt1480055/',
   'episodeAirDate': '2011-04-17',
   'episodeDescription': "Jon Arryn, the Hand of the King, is dead. King Robert Baratheon plans to ask his oldest friend, Eddard Stark, to take Jon's place. Across the sea, Viserys Targaryen plans to wed his sister to a nomadic warlord in exchange for an army.",
   'openingSequenceLocations': ["King's Landing",
    'Winterfell',
    'The Wall',
    'Pentos'],
   'scenes': [{'sceneStart': '0:00:40',
     'sceneEnd': '0:01:45',
     'location': 'The Wall',
     'subLocation': 'Castle Black',
     'characters': [{'name': 'Gared'},
      {'name': 'Waymar Royce'},
      {'name': 'Will'}]},
    {'sceneStart': '0:01:45',
     'sceneEnd': '0:03:24',
     'location': 'North of the Wall',
     'subLocation': 'The Haunted Forest',
     'characters': [{'name': 'Gared'},
      {'name': 'Waymar Royce'},
      {'name': 'Will'}]},
    {'sce

Nessa etapa descobrimos em quantos episódios cada personagem da série apareceu

In [17]:
episode_counter = Counter()
for episode in episodes_data['episodes']:
    characters = []
    for scene in episode['scenes']:
        for character in scene['characters']:
            characters.append(character['name'])
    characters = set(characters)

    for character in characters:
        episode_counter[character] += 1
episode_counter

Counter({'Arya Stark': 55,
         'Catelyn Stark': 26,
         'Tyrion Lannister': 63,
         'Illyrio Mopatis': 2,
         'Drogon': 30,
         'Jory Cassel': 5,
         'Jorah Mormont': 50,
         'Eddard Stark': 12,
         'Summer': 15,
         'Tommen Baratheon': 24,
         'Robert Baratheon': 7,
         'Cersei Lannister': 59,
         'Bran Stark': 37,
         'Wight Wildling Girl': 2,
         'Robb Stark': 23,
         'Shaggydog': 9,
         'Hodor': 23,
         'Sansa Stark': 56,
         'Jon Arryn': 1,
         'Qotho': 5,
         'Ghost': 21,
         'Nymeria': 3,
         'Jaime Lannister': 51,
         'White Walker': 14,
         'Joffrey Baratheon': 26,
         'Ros': 14,
         'Rhaegal': 26,
         'Viserys Targaryen': 5,
         'Theon Greyjoy': 45,
         'Rodrik Cassel': 12,
         'Sandor Clegane': 35,
         'Myrcella Baratheon': 14,
         'Maester Luwin': 14,
         'Jon Snow': 58,
         'Will': 1,
         'Gared': 1,


Montamos um dicionário com o nome do personagem e o número de aparições, descoberto anteriormente

In [18]:
training_set_part2 = []
for character_name, episode_appearance_count in episode_counter.items():
    training_row = OrderedDict()

    training_row['character_name'] = str(character_name)
    training_row['episode_appearance_count'] = episode_appearance_count

    training_set_part2.append(training_row)
training_set_part2

[OrderedDict([('character_name', 'Arya Stark'),
              ('episode_appearance_count', 55)]),
 OrderedDict([('character_name', 'Catelyn Stark'),
              ('episode_appearance_count', 26)]),
 OrderedDict([('character_name', 'Tyrion Lannister'),
              ('episode_appearance_count', 63)]),
 OrderedDict([('character_name', 'Illyrio Mopatis'),
              ('episode_appearance_count', 2)]),
 OrderedDict([('character_name', 'Drogon'), ('episode_appearance_count', 30)]),
 OrderedDict([('character_name', 'Jory Cassel'),
              ('episode_appearance_count', 5)]),
 OrderedDict([('character_name', 'Jorah Mormont'),
              ('episode_appearance_count', 50)]),
 OrderedDict([('character_name', 'Eddard Stark'),
              ('episode_appearance_count', 12)]),
 OrderedDict([('character_name', 'Summer'), ('episode_appearance_count', 15)]),
 OrderedDict([('character_name', 'Tommen Baratheon'),
              ('episode_appearance_count', 24)]),
 OrderedDict([('character_name',

Unimos os dois dicionários, obtendo o equivalente ao nosso treino

In [21]:
training_set = []

for character_row in training_set_part1:
    for episode_row in training_set_part2:
        if character_row['character_name'] == episode_row['character_name']:
            training_row = OrderedDict()

            training_row['character_name'] = str(character_row['character_name'])
            training_row['episode_appearance_count'] = episode_row['episode_appearance_count']
            training_row['kill_count'] = character_row['kill_count']
            training_row['dead'] = character_row['dead']

            training_set.append(training_row)

            break
            
training_set

[OrderedDict([('character_name', 'Addam Marbrand'),
              ('episode_appearance_count', 1),
              ('kill_count', 0),
              ('dead', 0)]),
 OrderedDict([('character_name', 'Aeron Greyjoy'),
              ('episode_appearance_count', 2),
              ('kill_count', 0),
              ('dead', 0)]),
 OrderedDict([('character_name', 'Akho'),
              ('episode_appearance_count', 3),
              ('kill_count', 0),
              ('dead', 1)]),
 OrderedDict([('character_name', 'Alliser Thorne'),
              ('episode_appearance_count', 19),
              ('kill_count', 1),
              ('dead', 1)]),
 OrderedDict([('character_name', 'Alton Lannister'),
              ('episode_appearance_count', 3),
              ('kill_count', 0),
              ('dead', 1)]),
 OrderedDict([('character_name', 'Alys Karstark'),
              ('episode_appearance_count', 4),
              ('kill_count', 0),
              ('dead', 0)]),
 OrderedDict([('character_name', 'Amory Lorc

Salvamos o dicionário como um CSV

In [22]:
with open(PATH_INPUT+'training-data.csv', 'w') as output_file:
    writer = csv.DictWriter(output_file, training_set[0].keys())
    writer.writeheader()
    writer.writerows(training_set)